In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pre_processing as pp
import analysis as anl
import pca

category = 'books'

hNeg = True #if true, add negative bigrams for negative reviews
noun = False #if true, add nouns

X, Y, vocabulary = pp.bow(category, hNeg, noun)

print("Vocabulário possui " + str(len(vocabulary)) + " palavras!")

Vocabulário possui 10768 palavras!


In [2]:
def normalizar(X):

    m, n = X.shape # m = qtde de objetos e n = qtde de atributos por objeto
    
    # Inicializa as variaves de saída
    X_norm = np.zeros( (m,n) ) #inicializa X_norm (base normalizada)
    #X_norm = lil_matrix((m, n))
    mu = 0 # inicializa a média
    sigma = 1 # inicializa o desvio padrão
      
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0, ddof=1)
    
    for i in range(m):
        for j in range(n):
            X_norm[i][j] = (X[i][j] - mu[j])/(sigma[j])
        
    return X_norm

Separa os dados em treinamento e teste:

In [3]:
# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X2, Y2 = X[idx_perm, :], Y[idx_perm]

#X2, Y2 = X[idx_perm, :], Y[idx_perm]

pTrain = 0.8

train_index, test_index = anl.stratified_holdOut(Y, pTrain)

Xtrain, Xval = X2[train_index, :], X2[test_index, :]
Ytrain, Yval = Y2[train_index], Y2[test_index]

Seleciona features com chi-quadrado (a partir dos dados de treinamento):

In [4]:
Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
Xval = Xval[:, index]

Xtrain = Xtrain.toarray()
Xval = Xval.toarray()

In [5]:
print("Número de features antes do chi-quadrado: " + str(len(vocabulary)))
print("----------------------------------------")
print("Número de features após chi-quadrado: " + str(len(new_vocabulary)))
print(new_vocabulary)

Número de features antes do chi-quadrado: 10768
----------------------------------------
Número de features após chi-quadrado: 723
['dna', 'distract', ('not', 'tri'), ('not', 'learn'), ('not', 'go'), 'bias', 'fred', 'shade', 'pathet', 'shallow', 'mayb', 'quot', 'pay', 'orwel', 'name', 'glimps', 'paragraph', 'extremist', ('never', 'seem'), ('not', 'rememb'), 'rule', 'lesson', 'anna', 'distort', 'juda', 'hesit', 'closer', ('not', 'take'), 'repress', 'authorit', 'mere', 'perhap', 'unbeliev', 'post', 'honestli', 'danger', 'learn', 'wast', 'sorri', 'horribl', 'neil', 'memori', 'bad', 'wonder', 'bother', 'mildli', 'kelsey', 'twentieth', ('not', 'found'), 'hendrix', 'memoir', 'attend', 'kagan', 'cold', 'addict', 'annoy', 'stair', 'stimul', 'commerci', 'much', 'adopt', 'lake', 'overr', 'vicari', 'royal', 'uniqu', 'soon', 'better', 'happili', 'improv', 'save', 'sever', 'nam', ('not', 'want'), 'predict', 'sick', ('not', 'need'), ('not', 'tell'), 'access', 'dull', 'unsympathet', 'polem', 'map', '

In [6]:
def distancia(x, X):
               
    m = X.shape[0] 
    D = np.zeros(m) 

    i = 0
    for amostra in X:
        D[i] = np.linalg.norm(amostra - x)
        i += 1
                    
    return D

In [7]:
def knn(x, X, Y, K):
        
    y = 0 
    
    ind_viz = np.ones(K, dtype=int)
    
    D = distancia(x, X)
    
    votos = np.zeros(len(set(Y)))

    ind_viz = np.argsort(D)[:K]
    
    for indice in ind_viz:
        votos[Y[indice]] += 1 
    y = np.argmax(votos)
        
    return y

In [8]:
def predicao(K, Xtrain, Ytrain, Xval, Yval):
    
    Ypred = []

    for i in range(Xval.shape[0]):
        y = knn(Xval[i], Xtrain, Ytrain, K)
        Ypred.append(y)
        
    acuracia = np.sum(Ypred==Yval)/len(Yval)
    
    return Ypred, acuracia


classes = np.unique(Y)

for k in range(3, 12, 2):
    print("K = " + str(k))
    Ypred, acuracia = predicao(k, Xtrain, Ytrain, Xval, Yval)
    print(acuracia)
    auxResults = anl.relatorioDesempenho(Yval, Ypred, classes, imprimeRelatorio=True)

K = 3
0.67

	Revocacao   Precisao   F-medida   Classe
	0.449       0.829      0.582      0
	0.903       0.609      0.727      1
	------------------------------------------------
	0.676       0.719      0.697      Média macro
	0.670       0.670      0.670      Média micro

	Acuracia: 0.670
K = 5
0.6725

	Revocacao   Precisao   F-medida   Classe
	0.429       0.863      0.573      0
	0.928       0.607      0.734      1
	------------------------------------------------
	0.679       0.735      0.706      Média macro
	0.672       0.672      0.672      Média micro

	Acuracia: 0.672
K = 7
0.675

	Revocacao   Precisao   F-medida   Classe
	0.410       0.903      0.564      0
	0.954       0.606      0.741      1
	------------------------------------------------
	0.682       0.755      0.716      Média macro
	0.675       0.675      0.675      Média micro

	Acuracia: 0.675
K = 9
0.6625

	Revocacao   Precisao   F-medida   Classe
	0.376       0.917      0.533      0
	0.964       0.595      0.736     

In [9]:
def curva_aprendizado(X, Y, Xval, Yval):
   
    """
    Funcao usada gerar a curva de aprendizado.
  
    Parametros
    ----------
  
    X : matriz com os dados de treinamento
  
    Y : vetor com as classes dos dados de treinamento
  
    Xval : matriz com os dados de validação
  
    Yval : vetor com as classes dos dados de validação
  
    """
    
    k = 5

    # inicializa as listas que guardarao a performance no treinamento e na validacao
    perf_train = []
    perf_val = []

    classes = np.unique(Y)
    
    for i in range(10, len(Y)):
        
        Ypred, acuracia = predicao(k, X[:i], Y[:i], X[:i], Y[:i])
        perf_train.append(acuracia)

        Ypred, acuracia = predicao(k, X[:i], Y[:i], Xval, Yval)
        perf_val.append(acuracia)


    ##################################################################################
       
    # Define o tamanho da figura 
    plt.figure(figsize=(20,12))

    # Plota os dados
    plt.plot(perf_train, color='blue', linestyle='-', linewidth=1.5, label='Treino') 
    plt.plot(perf_val, color='red', linestyle='-', linewidth=1.5, label='Validação')

    # Define os nomes do eixo x e do eixo y
    plt.xlabel(r'# Qtd. de dados de treinamento',fontsize='x-large') 
    plt.ylabel(r'Acuracia',fontsize='x-large') 

    # Define o título do gráfico
    plt.title(r'Curva de aprendizado', fontsize='x-large')

    # Acrescenta um grid no gráfico
    plt.grid(axis='both')

    # Plota a legenda
    plt.legend()
    
    plt.show()

In [10]:
#curva_aprendizado(Xtrain, Ytrain, Xval, Yval)

In [11]:
#K-folds

import k_folds as kf

#Pega todos os tipos de classes 
classes = classes = np.unique(Y)

# semente usada na randomizacao dos dados.
randomSeed = 10 

# gera os indices aleatorios que irao definir a ordem dos dados
idx_perm = np.random.RandomState(randomSeed).permutation(range(len(Y)))

# ordena os dados de acordo com os indices gerados aleatoriamente
X3, Y3 = X[idx_perm, :], Y[idx_perm]

# separa os dados em k folds
nFolds = 5
folds = kf.stratified_kfolds(Y3, nFolds, classes)

kfolds = 1
resultados=[[] for i in range(5)] # cria uma lista vazia para guardar os resultados obtidos em cada fold
for train_index, test_index in folds:

    print('\n-----------\n%d-fold: \n-----------\n' % (kfolds) )
    
    # se train_index ou test_index forem vazios, interrompe o laco de repeticao
    if len(train_index)==0 or len(test_index)==0: 
        print('\tErro: o vetor com os indices de treinamento ou o vetor com os indices de teste esta vazio')      
        break
        
    totalFold = len(train_index)+len(test_index)
    
    Xtrain, Xtest = X3[train_index, :], X3[test_index, :];
    Ytrain, Ytest = Y3[train_index], Y3[test_index];
    
    Xtrain, new_vocabulary, index = pp.chi2(Xtrain, Ytrain, vocabulary)
    Xtest = Xtest[:,index]

    Xtrain = Xtrain.toarray()
    Xtest = Xtest.toarray()
    
    i = 0
    for k in range(3, 12, 2):
        print("K = " + str(k))
        Ypred, acuracia = predicao(k, Xtrain, Ytrain, Xtest, Ytest)
        print(acuracia)
        auxResults = anl.relatorioDesempenho(Ytest, Ypred, classes, imprimeRelatorio=True)
        # adiciona os resultados do fold atual na lista de resultados
        resultados[i].append( auxResults )
        print(i)
        i+=1
        if( i == 5):
            i = 0
    

        
    kfolds+=1
    i = 0
for j in range(3, 12, 2) :
    print("K = " + str(j))
    kf.mediaFolds( resultados[i], classes )
    i+=1


-----------
1-fold: 
-----------

K = 3
0.6825

	Revocacao   Precisao   F-medida   Classe
	0.480       0.807      0.602      0
	0.885       0.630      0.736      1
	------------------------------------------------
	0.682       0.718      0.700      Média macro
	0.682       0.682      0.682      Média micro

	Acuracia: 0.682
0
K = 5
0.695

	Revocacao   Precisao   F-medida   Classe
	0.500       0.820      0.621      0
	0.890       0.640      0.745      1
	------------------------------------------------
	0.695       0.730      0.712      Média macro
	0.695       0.695      0.695      Média micro

	Acuracia: 0.695
1
K = 7
0.7

	Revocacao   Precisao   F-medida   Classe
	0.505       0.828      0.627      0
	0.895       0.644      0.749      1
	------------------------------------------------
	0.700       0.736      0.717      Média macro
	0.700       0.700      0.700      Média micro

	Acuracia: 0.700
2
K = 9
0.7175

	Revocacao   Precisao   F-medida   Classe
	0.525       0.854      0.650  